In [1]:
# system
import os
import shutil
import sys
import time

# math
import math
import numpy as np
import torch
# python image library
from PIL import Image as img

# OpenGL
from OpenGL.GLUT import *
from OpenGL.GLU  import *
from OpenGL.GL   import *

# utilities
sys.path.append( './util' )
from util.setup import *
import world
import ratbot
import opengl_text as text

import ratlab
from ratlab import *
from ratlab import __setupGlut__, __setupOpenGL__, __display__

Build a system with 3 basic modular,  sample episodes, estimate td, learning  

In [3]:
func=None
dim=None
speed=None
box=None
bar=None
goal=None
limit=50
wall_offset=None
touch_offset=None

In [ ]:
# the first step, sample episodes 
if( os.path.isdir('./save') == False ):
    os.mkdir('./save')
else:
    shutil.rmtree('./save')
    os.mkdir('./save') 
#if( os.path.exits('./save/exp_trajectory.txt') == True ):
#    os.remove('./save/exp_trajectory.txt')

if dim != None:                # [10,10,100]
    ctrl.setup.world.dim = numpy.array(dim)    
if speed != None:              # 1.0
    ctrl.setup.rat.speed = float(speed)   
if box != None:                # [1, 1]
    ctrl.setup.world.obstacles.append( [float(box[0]), float(box[1]), float(box[1])] ) # obstacles' number, side_length, side_width
if bar != None:    
    ctrl.setup.world.obstacles.append( [float(bar[0]), float(bar[1])] ) # obstacles' number, width
if goal != None:               # [4,5,2]
    center_x = float(goal[0])
    center_y = float(goal[1])
    radius = float(goal[2])
    region_x = range(int(center_x - radius), int(center_x + radius + 1))
    region_y = range(int(center_x - radius), int(center_x + radius + 1))
    for m in region_x:
        for n in region_y:
            ctrl.setup.world.goals.append( [m, n] ) # x, y     
if limit != None:              # 1000
    ctrl.config.limit = float(limit)
    ctrl.options.show_progress = True
if wall_offset != None:             # 1
    ctrl.setup.world.wall_offset = float(wall_offset)    # the safe distance to avoid hitting a wall
if touch_offset != None:             # 2
    ctrl.setup.world.touch_offset = float(touch_offset)  # within the distance from a wall, providing a touch signal

ctrl.setup.toFile('./save/exp_setup')

__setupGlut__()
__setupOpenGL__()
# setup world and agent  
ctrl.modules.world = world.World( ctrl.setup.world )

ctrl.state.initPos = ctrl.modules.world.randomPosition()
ctrl.modules.rat = ratbot.RatBot( ctrl.state.initPos, ctrl )

rnn = RNN.RNN(4, 2, 512, 8)
print ('size',rnn.i2h.size(), rnn.a2h.size())
# action loop      
hidden = rnn.initHidden()
#     Hiddens = []
while True:
    #touch 1-west; 2-south; 3-east; 4-north      
    __display__()
    if func == None:
        touch = torch.eye(4)[ctrl.save.touch - 1]
        act = torch.from_numpy(ctrl.save.act).type(torch.FloatTensor)
#             print (touch.size(), touch.size(), act.size())
        out, hidden = rnn(touch, hidden, act) # ctrl.save.act [0,1] = 90 degree = North
#             Hiddens.append(hidden)
        act_ = numpy.argmax(out.cpu().data.numpy().squeeze())
        act = ctrl.table.sample_dir[act_][1]
#             print ('output', act_, act)
        ctrl.state.initAction = [float(act[0]), float(act[1])]

size torch.Size([4, 512]) torch.Size([2, 512])


In [ ]:
# the second step, evaluate td  , readout the action vision, hidden states replay and estimate td error
def TD(self, Q_now, Q_next, action, reward):
    # target Q is for state before updated, it only needs to update the value assocate with action taken
    targetQ = Q_now.clone().detach()
    # new Q attached with the new state
    # max of q for calculating td error
    Qmax = torch.max(Q_next)
    delta  =  torch.FloatTensor([reward]).cuda(self.gpu) + self.discount*Qmax  - targetQ[0, action]

    # eligilibty trace for updating all last states before because of the information about new state
    self.trace = [e * self.discount * self.lam for e in self.trace]
    # eligibility trace attach new state
    self.Qs.append(targetQ)
    self.trace.append(1)
    # corresponding features h
    # update all last action values with eligibility trace, the q will add a new updated value
    def f(e, delta, q):
#             print (e, delta, q)
        q[0, action] = q[0, action] + self.alpha * delta * e
        return q
#         print (self.trace)
    self.Qs = [f(e, delta, q) for e, q in zip(self.trace, self.Qs)]

# turn histroy of data into a target vector, the action is this step chosen action, so action1, the reward is this stepreward, so the reward0
def Value_backward(self, Predicts, Actions, Rewards):
    self.trace = []
    # print (len(Actions), len(Rewards), len(Predicts))
    for Q_now, Q_next, action, reward in zip(Predicts[:-1], Predicts[1:], Actions[1:], Rewards):
        action = np.int(torch.argmax(action).cpu().data.numpy())
        # print (action)
        self.TD(Q_now, Q_next, action, reward)

In [ ]:
# the third step learning.   
  def train_sgd(self, lr_rate=1e-5, batchsize = 10):
        # it is important to keep readout update quick 
        if self.Net == 'RNN':
            Optimizer = torch.optim.Adam(
                [
                    {'params': self.net.i2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bh, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2o, 'lr': 10 * lr_rate, 'weight_decay': 0},
                    {'params': self.net.bo, 'lr': 10 * lr_rate, 'weight_decay': 0},
                ]
            )
        elif self.Net == 'GRU':
            Optimizer = torch.optim.Adam(
                [
                    {'params': self.net.i2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.i2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bh, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bf, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2o, 'lr': 10 * lr_rate, 'weight_decay': 0},
                    {'params': self.net.bo, 'lr': 10 * lr_rate, 'weight_decay': 0},
                ]
            )
        elif self.Net == 'LSTM':
                Optimizer = torch.optim.Adam(
                    [
                        {'params': self.net.i2h, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.a2h, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.h2h, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.bh, 'lr': lr_rate, 'weight_decay': 0},
                        # forget gate
                        {'params': self.net.i2f, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.a2f, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.h2f, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.bf, 'lr': lr_rate, 'weight_decay': 0},
                        # input gate
                        {'params': self.net.i2i, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.a2i, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.h2i, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.bi, 'lr': lr_rate, 'weight_decay': 0},
                        # output gate
                        {'params': self.net.i2out, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.a2out, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.h2out, 'lr': lr_rate, 'weight_decay': 0},
                        {'params': self.net.bout, 'lr': lr_rate, 'weight_decay': 0},
                        # readout
                        {'params': self.net.h2o, 'lr': 10 * lr_rate, 'weight_decay': 0},
                        {'params': self.net.bo, 'lr': 10 * lr_rate, 'weight_decay': 0},
                    ]
                )

        # read N = epoch histories of data, compute values functions, all hidden states, and do back prob
        Data = [(v, a, r, h, cell, c) for v, a, r, h, cell, c in zip(self.Visions_batch, self.Actions_batch, self.Rewards_batch , self.Hiddens0, self.Cells0, self.Contexts)]
        # sample history one by one
        data_list = np.random.randint(len(Data), size = batchsize)
        Loss = 0
        for r in data_list:
            # reconstruct hidden states
            # hidden0 = torch.randn(1, 512).cuda(self.gpu)
            # update q target function with target net
            Visions, Actions, Rewards, hidden0, cell0, context = Data[r]
            Predicts_target = self.net_target.forward_sequence_values(Visions, Actions, hidden0, cell0, context)
            Predicts = self.net.forward_sequence_values(Visions, Actions, hidden0, cell0, context)
            # backward to compute values, but the backward values should be from last state
            self.Value_backward(Predicts_target, Actions, Rewards)
            if len(self.Qs)>=2:
                loss = torch.mean((Predicts[:-1] - torch.stack(self.Qs)) ** 2)
                loss.backward()
            if len(self.Qs) >= 2:
                Loss += loss
            self.Qs = []
            for p, name in zip([self.net.i2h, self.net.a2h, self.net.h2h, self.net.bh], ['i2h','a2h', 'h2h', 'bh']):
                p.grad.data.clamp_(-5, 5)
            Optimizer.step()
            self.net.zero_grad()
            # update target network
            for name in self.net.state_dict():
                weight = 0.1 * self.net.state_dict()[name].data  + 0.9 * self.net_target.state_dict()[name].data
                self.net_target.state_dict()[name].data.copy_(weight)
                    # print (name, torch.norm(self.net_target.state_dict()[name] - self.net.state_dict()[name]))
        print('loss', Loss)

    def train_sgd_asyn(self, lr_rate=1e-5, batchsize=10):
        # it is important to keep readout update quick
        if self.Net == 'RNN':
            Optimizer = torch.optim.Adam(
                [
                    {'params': self.net.i2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bh, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2o, 'lr': 10 * lr_rate, 'weight_decay': 0},
                    {'params': self.net.bo, 'lr': 10 * lr_rate, 'weight_decay': 0},
                ]
            )
        elif self.Net == 'GRU':
            Optimizer = torch.optim.Adam(
                [
                    {'params': self.net.i2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.i2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bh, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bf, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2o, 'lr': 10 * lr_rate, 'weight_decay': 0},
                    {'params': self.net.bo, 'lr': 10 * lr_rate, 'weight_decay': 0},
                ]
            )
        elif self.Net == 'LSTM':
            Optimizer = torch.optim.Adam(
                [
                    {'params': self.net.i2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2h, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bh, 'lr': lr_rate, 'weight_decay': 0},
                    # forget gate
                    {'params': self.net.i2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2f, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bf, 'lr': lr_rate, 'weight_decay': 0},
                    # input gate
                    {'params': self.net.i2i, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2i, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2i, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bi, 'lr': lr_rate, 'weight_decay': 0},
                    # output gate
                    {'params': self.net.i2out, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.a2out, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.h2out, 'lr': lr_rate, 'weight_decay': 0},
                    {'params': self.net.bout, 'lr': lr_rate, 'weight_decay': 0},
                    # readout
                    {'params': self.net.h2o, 'lr': 10 * lr_rate, 'weight_decay': 0},
                    {'params': self.net.bo, 'lr': 10 * lr_rate, 'weight_decay': 0},
                ]
            )

        Data = [(v, a, r, h, c) for v, a, r, h, c in
                zip(self.Visions_batch, self.Actions_batch, self.Rewards_batch, self.Hiddens0, self.Contexts)]
        # sample history one by one
        # Trainloader = torch.utils.data.dataloader(Data, shuffle = True)
        # get predicts and targets
        Predicts_batch = []
        Targets_batch = []
        for data in Data:
            # reconstruct hidden states
            # hidden0 = torch.randn(1, 512).cuda(self.gpu)
            # update q target function with target net
            Visions, Actions, Rewards, hidden0, context = data
            # Predicts_target = self.net_target.forward_sequence_values(Visions, Actions, hidden0, context)
            Predicts = self.net.forward_sequence_values(Visions, Actions, hidden0, context)
            # backward to compute values, but the backward values should be from last state
            self.Value_backward(Predicts, Actions, Rewards)
            Predicts_batch.extend(Predicts[:-1])
            Targets_batch.extend(torch.stack(self.Qs))
            self.Qs = []
        # compute loss
        Loss = 0
        traindata = [(predict, target) for predict, target in zip(Predicts_batch, Targets_batch)]
        Trainloader = torch.utils.data.DataLoader(traindata, shuffle = True)
        for i, data in enumerate(Trainloader):
            predict, target = data
            loss = torch.sum((predict - target) ** 2)
            loss.backward(retain_graph = True)
            Loss += loss
        for p, name in zip([self.net.i2h, self.net.a2h, self.net.h2h, self.net.bh], ['i2h', 'a2h', 'h2h', 'bh']):
            p.grad.data.clamp_(-5, 5)
        Optimizer.step()
        self.net.zero_grad()
            # update target network
                # print (name, torch.norm(self.net_target.state_dict()[name] - self.net.state_dict()[name]))
        print('loss', Loss)

    # save the data in batch and pass to network
    def experiment(self, epochs = 10, epsilon=0, reward_control=None, train_hidden = True,
                   decode=False, size_range=[10], test=False, lr_rate = 1e-5, implicit = False, update_batch = True, train_method = 'train_sgd'):
        # initialize, might take data during test

        if update_batch == True:
            for i in range(1):
                r = np.random.randint(len(self.Actions_batch), size=1)[0]
                self.Actions_batch.pop(r)
                self.Visions_batch.pop(r)
                self.Rewards_batch.pop(r)
                self.Contexts.pop(r)
                self.Hiddens0.pop(r)
                self.Cells0.pop(r)
        self.episode(epochs=epochs, epsilon=epsilon, reward_control=reward_control, size_range=size_range,
                     train_hidden=train_hidden, test=test,
                     decode=decode, implicit=implicit)
        func = getattr(self, train_method)
        func(lr_rate = lr_rate)
        self.Targets_batch = []
        self.Pos_batch = []
        process = psutil.Process(os.getpid())
        print('clear session data', process.memory_info().rss)
